In [1]:
from prepare_data import preprocess_data_czech
from field_info import FieldInfo
from tensor_encoder import TensorEncoder
import pandas as pd
import tensorflow as tf
import numpy as np

2023-12-03 10:25:51.856680: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
raw_data = pd.read_csv('../DATA/tr_by_acct_w_age.csv')
data, LOG_AMOUNT_SCALE, TD_SCALE, START_DATE, TCODE_TO_NUM, NUM_TO_TCODE = preprocess_data_czech(raw_data)
data2 = data[['account_id','age','tcode', 'tcode_num', 'datetime', 'month', 'dow', 'day', 'dtme' ,'log_amount_sc','td_sc']]
df= data2.copy()

n_tcodes = len(TCODE_TO_NUM)

info = FieldInfo(n_tcodes)

max_seq_len = 80
min_seq_len = 20
df_test = df[:10]
encoder = TensorEncoder(df, info, max_seq_len, min_seq_len)


In [3]:
encoder.encode()

2023-12-03 10:26:38.492478: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-12-03 10:26:38.493895: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-12-03 10:26:38.521152: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-12-03 10:26:38.521401: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:af:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-12-03 10:26:38.521435: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-12-03 10:2

Finished encoding 2000 of 14354 seqs
Finished encoding 4000 of 14354 seqs
Finished encoding 6000 of 14354 seqs
Finished encoding 8000 of 14354 seqs
Finished encoding 10000 of 14354 seqs
Finished encoding 12000 of 14354 seqs
Finished encoding 14000 of 14354 seqs
Took 87.45 secs


In [4]:
df

,account_id,age,tcode,tcode_num,datetime,month,dow,day,dtme,log_amount_sc,td_sc
0,1,29,CREDIT__CREDIT IN CASH__nan,0,1995-03-24,3,4,24,7,2.823750,0.000000
1,1,29,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1,1995-04-13,4,3,13,17,3.355869,3.298201
2,1,29,CREDIT__CREDIT IN CASH__nan,0,1995-04-23,4,6,23,7,3.858949,1.649100
3,1,29,CREDIT__nan__INTEREST CREDITED,2,1995-04-30,4,6,30,0,1.228484,1.154370
4,1,29,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1,1995-05-13,5,5,13,18,3.355869,2.143831
...,...,...,...,...,...,...,...,...,...,...,...
1056315,11382,46,DEBIT__CASH WITHDRAWAL__nan,3,1998-12-02,12,2,2,29,4.148672,0.329820
1056316,11382,46,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1,1998-12-10,12,3,10,21,4.390393,1.319280
1056317,11382,46,DEBIT__CASH WITHDRAWAL__nan,3,1998-12-25,12,4,25,6,3.575677,2.473651
1056318,11382,46,CREDIT__nan__INTEREST CREDITED,2,1998-12-31,12,3,31,0,2.347677,0.989460


In [4]:
from field_info import FieldInfo
ACTIVATIONS = {
    "td_sc": "relu",
    "log_amount_sc": "relu"
}
fieldInfo = FieldInfo(n_tcodes)
config = {}
config["ORDER"] = fieldInfo.DATA_KEY_ORDER
config["FIELD_STARTS_IN"] = fieldInfo.FIELD_STARTS_IN
config["FIELD_DIMS_IN"] = fieldInfo.FIELD_DIMS_IN
config["FIELD_STARTS_NET"] = fieldInfo.FIELD_STARTS_NET
config["FIELD_DIMS_NET"] = fieldInfo.FIELD_DIMS_NET
config["ACTIVATIONS"] = ACTIVATIONS

In [35]:
 fieldInfo.FIELD_STARTS_TAR

{'tcode_num': 0,
 'dow': 1,
 'month': 2,
 'day': 3,
 'dtme': 4,
 'td_sc': 5,
 'log_amount_sc': 6}

In [6]:
fieldInfo.CLOCK_DIMS

{'day': 31, 'dtme': 31, 'dow': 7, 'month': 12}

In [16]:
fieldInfo.FIELD_DIMS_IN

{'tcode_num': 16,
 'dow': 2,
 'month': 2,
 'day': 2,
 'dtme': 2,
 'td_sc': 1,
 'log_amount_sc': 1}

In [5]:
encoder.count_variable_length_seqs_with_overlap(), encoder.count_seqs_with_overlap(), encoder.count_seqs_in_df()

(99101, 72764, 14354)

In [6]:
encoder.inp_tensor.shape

(14354, 81, 26)

In [ ]:
encoder.inp_tensor

In [7]:
# Create a TensorFlow dataset from the entire tensors
ds_all = tf.data.Dataset.from_tensor_slices((encoder.inp_tensor.astype(np.float32), encoder.tar_tensor.astype(np.float32)))

def make_batches(ds, buffer_size, batch_size):
    return ds.cache().shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

BUFFER_SIZE = ds_all.cardinality().numpy()
bs = 64  # batch size

# Create train batches from the entire dataset
train_batches = make_batches(ds_all, BUFFER_SIZE, bs)

In [9]:
train_batches

<PrefetchDataset shapes: ((None, 81, 26), (None, 80, 7)), types: (tf.float32, tf.float32)>

In [8]:
# create only one sample of inp and tar to trace one iteration of loop
j = 0
for (batch_no, (inp, tar)) in enumerate(train_batches):
    if j == 0:
       break

In [22]:
tar[0,0]

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([ 3.       ,  1.       ,  1.       ,  3.       , 28.       ,
        0.       ,  2.9893363], dtype=float32)>

In [9]:
from modules import InputEmbedLayer,InputEmbedLayer_Res, ResidualLayer, RandomNoise_Simulator_Normal, positional_encoding, MultiHeadAttention, Transformer, create_masks, DecoderLayer, Decoder
import tensorflow as tf
features = 26
d_embedding = 128
dff = 128
d_model = 128
batch_size = 64
seq_len = 80
maximum_position_encoding = 256
rate = 0.1
num_heads = 2
num_layers = 4
deepmodel = Transformer(features, dff, d_embedding, d_model, maximum_position_encoding,num_heads, num_layers,config, rate=0.1)

In [10]:
transformer  = deepmodel.make_transformer(tar, inp)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [11]:
preds = transformer(inp[:, :-1])

2023-12-03 10:37:08.766132: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [17]:
preds

{'tcode_num': <tf.Tensor: shape=(64, 80, 16), dtype=float32, numpy=
 array([[[-0.92637163, -1.3785548 ,  0.5696109 , ...,  2.2086666 ,
          -0.36722243, -1.6487352 ],
         [-0.20490918, -0.842854  ,  0.4116467 , ...,  2.976397  ,
           0.7901808 , -0.05809675],
         [ 0.06890717, -0.69596756, -0.8012815 , ...,  2.2384899 ,
           1.0349656 ,  0.11382397],
         ...,
         [ 0.61111605, -1.8831675 , -1.118025  , ...,  1.4789788 ,
           0.66860986, -1.5804174 ],
         [ 0.12852155, -1.1900693 , -0.64241785, ...,  2.4992893 ,
          -0.7901032 , -0.10753079],
         [ 0.9642946 ,  0.02653459, -0.5988542 , ...,  2.4711058 ,
          -1.542403  ,  0.26788515]],
 
        [[-0.27986443, -1.9937177 ,  0.15879466, ...,  2.4494317 ,
           0.7124766 , -1.3402632 ],
         [ 0.2802798 , -0.1582319 ,  0.3151046 , ...,  2.7751892 ,
           0.7976539 , -0.17377616],
         [ 0.8318278 , -1.2953007 ,  0.47676098, ...,  2.4675002 ,
          -0.217

In [12]:
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 26)]   0                                            
__________________________________________________________________________________________________
input_embed_layer (InputEmbedLa (None, None, 128)    19968       input_1[0][0]                    
__________________________________________________________________________________________________
tf.compat.v1.shape (TFOpLambda) (3,)                 0           input_embed_layer[0][0]          
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici ()                   0           tf.compat.v1.shape[0][0]         
______________________________________________________________________________________________

In [ ]:
tar

In [13]:
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return  -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi)

loss_scce_logit = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [18]:
loss_scce_logit = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
LOSS_WEIGHTS = {
 'td_sc':1.,
 'month': 0.015,
 'day': 0.025,
 'dtme': 0.025,
 'dow': 0.01,
 'tcode_num': 1.,
 'log_amount_sc': 2.}

FIELD_STARTS_TAR = fieldInfo.FIELD_STARTS_TAR
FIELD_DIMS_TAR = fieldInfo.FIELD_DIMS_TAR
LOSS_TYPES = fieldInfo.LOSS_TYPES

def loss_function(real, pred):
    loss_parts = []
    loss_parts_weighted = []
    mask = tf.math.logical_not(tf.math.equal(tf.reduce_sum(real, axis=2), 0))
    for k, k_pred in preds.items():
        st = FIELD_STARTS_TAR[k]
        end = st + FIELD_DIMS_TAR[k]
        loss_type = LOSS_TYPES[k]
        if loss_type == "scce":
           loss_ = loss_scce_logit(real[:, :, st:end], k_pred)
        elif loss_type == "pdf":
           loss2_ = -log_normal_pdf(real[:, :, st:end], k_pred[:,:,0:1], k_pred[:,:,1:2])[:,:,0]
        mask = tf.cast(mask, dtype=loss_.dtype)
        loss_ *= mask
        loss_ = tf.reduce_sum(loss_)/tf.reduce_sum(mask) 

        loss_parts.append(loss_)
        loss_parts_weighted.append(loss_ * LOSS_WEIGHTS[k])
    return tf.reduce_sum(loss_parts_weighted)

In [19]:
loss_function(tar, preds)

<tf.Tensor: shape=(), dtype=float32, numpy=16.30883>

In [14]:
from modules import InputEmbedLayer,InputEmbedLayer_Res, ResidualLayer, RandomNoise_Simulator_Normal, positional_encoding, MultiHeadAttention, create_masks, DecoderLayer, Decoder
import tensorflow as tf
features = 26
d_embedding = 128
dff = 128
d_model = 128
batch_size = 64
seq_len = 80
maximum_position_encoding = 256
rate = 0.1
num_heads = 2
num_layers = 4
z = RandomNoise_Simulator_Normal(batch_size, seq_len, features)

#Transformer Model
input_ = tf.keras.layers.Input(shape=(None, features))
x = InputEmbedLayer(features, dff , d_embedding)(input_)

pos_encoding = positional_encoding(maximum_position_encoding, d_embedding)   #(1, maximum_position_encoding=256, d_model=128)

seq_len = tf.shape(x)[1]
x += pos_encoding[:, :seq_len, :]     #x is the output of Input layer

x = tf.keras.layers.Dropout(rate)(x, training=True)

mask, _ = create_masks(tar)
d_inp_decoder = tf.keras.backend.int_shape(x)[-1]

out, attention_weights = Decoder(num_layers, d_inp_decoder, d_model, num_heads, dff)(x, True, mask)

final_output = tf.keras.layers.Dense(d_model, activation=None)(out)
model = tf.keras.models.Model(input_, final_output)


In [ ]:
model.summary()

In [18]:
# from modules import InputEmbedLayer,InputEmbedLayer_Res, ResidualLayer, RandomNoise_Simulator_Normal, positional_encoding, MultiHeadAttention, create_masks, DecoderLayer
# import tensorflow as tf
# features = 26
# d_embedding = 128
# dff = 128
# d_model = 128
# batch_size = 64
# seq_len = 80
# maximum_position_encoding = 256
# rate = 0.1
# num_heads = 2
# num_layers = 4
# z = RandomNoise_Simulator_Normal(batch_size, seq_len, features)

# #Transformer Model
# input_ = tf.keras.layers.Input(shape=(None, features))
# x = InputEmbedLayer(features, dff , d_embedding)(input_)

# pos_encoding = positional_encoding(maximum_position_encoding, d_embedding)   #(1, maximum_position_encoding=256, d_model=128)

# seq_len = tf.shape(x)[1]
# x += pos_encoding[:, :seq_len, :]     #x is the output of Input layer

# x = tf.keras.layers.Dropout(rate)(x, training=True)

# attention_weights = {}
# mask, _ = create_masks(tar)
# for i in range(num_layers):
#     d_inp_decoder = tf.keras.backend.int_shape(x)[-1]
#     x, attentionweights = DecoderLayer(d_inp_decoder, d_model, num_heads, dff)(x, True, mask)
#     attention_weights['decoder_layer{}'.format(i+1)] = attentionweights

# final_output = tf.keras.layers.Dense(d_model, activation=None)(x)


# model = tf.keras.models.Model(input_, final_output)


In [ ]:
model(inp[:, :-1])

In [17]:
attention_weights

{'decoder_layer1': <KerasTensor: shape=(64, 2, 80, 80) dtype=float32 (created by layer 'decoder_3')>,
 'decoder_layer2': <KerasTensor: shape=(64, 2, 80, 80) dtype=float32 (created by layer 'decoder_3')>,
 'decoder_layer3': <KerasTensor: shape=(64, 2, 80, 80) dtype=float32 (created by layer 'decoder_3')>,
 'decoder_layer4': <KerasTensor: shape=(64, 2, 80, 80) dtype=float32 (created by layer 'decoder_3')>}